# Control the robot_plotter

## Import all the libraries
* The `serial` library is used to communicate.
* The `sleep` command from the `time` library is used to add delays in the execution.
* The `svgpathtools` library is used to read a vector drawing file.
* The `pandas` library is used to organize the coordinates and angle data.
* The `numpy` library is used to perform the necessary calculations.
* The `matplotlib` library is used to plot the coordinates to confirm if they are the correct ones.

In [ ]:
import serial
from time import sleep
from svgpathtools import svg2paths, wsvg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Dimensions of the arms
The dimensions of the arms of the robot are set here. The exact value must be known for calculations of forward and inverse kinematics.


In [ ]:
l2 = 100 #mm
d = 80   #mm
l1 = 60  #mm

## Forward Kinematics
These are the equations that describe how the joint coordinates ($q_1 , q_2$) affect the end effector location ($x , y$).

In [ ]:
def forward_kinematics(row):
    q1, q2 = row[['q1', 'q2']]
    q1, q2 = np.deg2rad([q1, q2])
    
    A = l1*np.array([np.cos(q1), np.sin(q1)])
    B = np.array([d+l1*np.cos(q2), l1*np.sin(q2)])
    D = B-A
    
    psi = np.arctan2(D[1], D[0])
    h = np.linalg.norm(D)
    del1 = np.arccos(h/(2*l2))
    phi1 = del1+psi
    C = A+l2*np.array([np.cos(phi1), np.sin(phi1)])
    return C

## Inverse Kinematics
These are the equations that calculate the joint coordinates ($q_1 , q_2$) based on the desired end effector location ($x , y$).

In [ ]:
def inverse_kinematics(row):
    x, y = row[['x','y']]
    C = np.sqrt(x**2+y**2)
    e = np.sqrt((d-x)**2+y**2)
    q1 = np.arctan2(y,x)+np.arccos((l1**2+C**2-l2**2)/(2*l1*C))
    q2 = np.pi-np.arctan2(y,d-x)-np.arccos((l1**2+e**2-l2**2)/(2*l1*e))
    return np.rad2deg([q1, q2])

## Load the path
Here, we read the **svg** file that contains the drawing. The end effector must travel through all the points in order to create the drawing. Based on the desired end effector locations, the joint coordinates are calculated using the `inverse_kinematics` function.

In [ ]:
paths, attributes = svg2paths('drawings/engrave.svg')
result = []
for path in paths:
    for line in path:
        x = line.start.real
        y = line.start.imag
        result.append((x,y))

result = pd.DataFrame(result, columns= ['x', 'y'])
middle = np.mean([min(result['x']),max(result['x'])])
result['x'] -= middle
result['x'] = -result['x'] #Flip the drawing
result['x'] += 40 #
result['y'] += 60

result[['q1', 'q2']] = result.apply(inverse_kinematics, axis=1, result_type='expand')
#result = result.dropna()


## Plot the path
Here, we plot the path of the end effector to confirm if the joints are calculated correctly. We do this by applying the `forward_kinematics` function on the joint angles to get back the end effector position.

In [ ]:
result[['xd', 'yd']] = result.apply(forward_kinematics, axis=1, result_type='expand')
result.plot('xd', 'yd')
plt.axis('equal')

## Send joint coordinates to robot
Here, we send the calculated joint coordinates to the robot via the Serial port. Define the port name correctly.

**NOTE:** If you get an error opening the port, make sure the Serial Monitor is closed.

In [ ]:
with serial.Serial(port='COM12', baudrate=9600) as ser:
    x = input("Write H0 after homing the motors to their 0 angle positions.")
    ser.write(x.encode())
    sleep(3)
    
    print("Setting the motor speed.")
    command = f'S700\n'
    print(command)
    ser.write(command.encode())
    
    print("Sending joint coordinates to the robot.")
    for i, row in result.iterrows():

        x, y = (row[['q1', 'q2']]).astype(float)
        command = f'X{x:.3f}Y{y:.3f}\n'
        print(command)
        ser.write(command.encode())
        
        while ser.read().decode()!='G':
            pass

        
    x, y = [0,0]    
    command = f'X{x}Y{y}\n'
    print(command)
    ser.write(command.encode())